In [66]:
#Libraries
import os
import xml.etree.ElementTree as ET
import cv2
import numpy as np

In [67]:
# Specify the paths to the folders containing images and XML files
image_folder = r"C:\Dalia\Spheroids\PANC1\Annotations\01-Shahbaz-aftertraining\Image"
xml_folder = r"C:\Dalia\Spheroids\PANC1\Annotations\01-Shahbaz-aftertraining\XML"
save_folder = r"C:\Dalia\Spheroids\PANC1\Annotations\01-Shahbaz-aftertraining\output"

In [68]:
def draw_polygon_on_image(xml_file, image_file, save_folder, counter):
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Load the image
    image = cv2.imread(image_file)

    # Iterate over the object annotations
    for obj in root.iter("object"):
        # Extract the class label
        class_label = obj.find("name").text

        # Find the polygon points
        polygon = obj.find("polygon")
        if polygon is not None:
            # Get the list of points in the polygon
            points = []
            for point_index in range(1, len(polygon) + 1):
                x_tag = polygon.find(f"x{point_index}")
                y_tag = polygon.find(f"y{point_index}")

                if x_tag is not None and y_tag is not None:
                    x = x_tag.text
                    y = y_tag.text
                    points.append((x, y))
                else:
                    break

            # Convert the points to integer values
            points = [(int(float(x)), int(float(y))) for x, y in points]

            # Draw the polygon on the image
            points_array = np.array(points)
            cv2.polylines(image, [points_array], isClosed=True, color=(255,0, 0), thickness=2)

            # Put the class label text on the image
            #cv2.putText(image, class_label, (points[0][0], points[0][1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Save the image with the polygon
    save_path = os.path.join(save_folder, f"image_{counter}.jpg")
    cv2.imwrite(save_path, image)

In [69]:
    # Get the list of XML files
xml_files = [file for file in os.listdir(xml_folder) if file.endswith(".xml")]


In [70]:
    # Get the list of image files
image_files = [file for file in os.listdir(image_folder) if file.endswith(".jpg")]


In [71]:
# Iterate over the XML and image files simultaneously, call draw polygon
for counter, (image_file, xml_file) in enumerate(zip(sorted(os.listdir(image_folder)), sorted(os.listdir(xml_folder)))):
    # Create the full paths to the XML and image files
    xml_path = os.path.join(xml_folder, xml_file)
    image_path = os.path.join(image_folder, image_file)
    # Draw polygons on the image
    draw_polygon_on_image(xml_path, image_path, save_folder, counter)